### Change the 2D image to 3D volum to plot the FROC Curve
- get each image the slice info and as the third conditioin
- slice by slice to calute the overlap between the boxes, and set the superparameter: "a", as the overlap
value, when > "a", means they own to the same 3d box, then do the mark
- the same mark means the same 3d boxe, and transform the slices marked to a 3d boxes
    - the depth is len(slices)'
    - the width and the height is the min and max side of the 2d boxes of the slices

In [144]:
import numpy as np
import torchio as tio
import pandas as pd
import csv
import json

In [145]:
def get_id(path_coco):
    # 打开并读取JSON文件
    with open(path_coco, 'r') as file:
        data = json.load(file)
    
    images = data['images']
    # 用于存储需要修改的image_id
    image_names = []
    image_ids = []
    # 查找所有positive为0的图像的ID
    for image in images:
        image_names.append(image['file_name'])
        image_ids.append(image['id'])
    return image_names, image_ids

path_coco = '/public_bme/data/xiongjl/Deformable-DETR_data_public/coco/annotations/instances_val2017_alllayers_val.json'
print(get_id(path_coco))

In [146]:
import pandas as pd
from collections import defaultdict

def get_dict(image_name_given, type_name='pred'):
    csv_file = '/public_bme/data/xiongjl/Deformable-DETR_public/csv_file/ddetr_predict_alllayers.csv'
    
    # 读取CSV文件并进行初步过滤
    df = pd.read_csv(csv_file)
    df_filtered = df[(df['type_name'] == type_name) & 
                     (df['label'] == 1) & 
                     (df['score'] >= 0.005)]

    # 进一步筛选符合条件的行
    df_filtered = df_filtered[df_filtered['image_name'].str.contains(image_name_given)]

    # 初始化数据字典
    data_dict = defaultdict(list)

    # 遍历过滤后的DataFrame并组织到字典中
    for index, row in df_filtered.iterrows():
        image_name = row['image_name']
        slice_num = image_name.split("_")[1]
        box_info = [row['score'], row['x1'], row['y1'], row['x2'], row['y2']]
        data_dict[slice_num].append(box_info)

    return data_dict

# 调用函数并打印结果
dict_data = get_dict('02191204216120')
print(dict_data)


defaultdict(<class 'list'>, {'6': [[0.04371, 232.87, 298.83, 243.11, 305.73]], '24': [[0.00773, 26.57, 137.67, 35.13, 149.19]], '56': [[0.03555, 236.62, 308.99, 245.7, 317.41]], '57': [[0.64885, 230.8, 309.5, 240.36, 317.79]], '101': [[0.99652, 230.08, 271.7, 241.52, 278.65]], '102': [[0.97974, 229.6, 271.56, 240.54, 278.25]], '103': [[0.92319, 230.68, 272.27, 242.63, 278.42]], '104': [[0.78183, 235.07, 271.58, 247.66, 277.81]], '105': [[0.31972, 241.33, 271.13, 252.75, 277.94]], '106': [[0.25254, 243.61, 269.38, 253.86, 275.98]], '107': [[0.04613, 247.35, 283.63, 257.34, 291.06]], '108': [[0.96262, 244.96, 268.78, 253.87, 276.57], [0.05772, 245.21, 265.76, 253.08, 272.96]], '109': [[0.70568, 244.4, 269.99, 253.15, 278.21]], '110': [[0.94263, 245.28, 269.1, 253.98, 276.62]], '111': [[0.01688, 222.74, 288.91, 233.49, 297.29]], '113': [[0.2215, 243.28, 270.5, 252.83, 279.04]], '139': [[0.04387, 123.9, 242.9, 132.42, 249.92]], '145': [[0.58737, 289.24, 192.37, 299.14, 200.36]], '149': [[0

### Return the 2d boxes dict data of one nii image to 3d boxes

In [148]:
import copy

def merge_to_3d_boxes(data_dict, overlap_threshold=0.5):
    # 初始化一个空列表，用于存储合并后的3D boxes
    merged_3d_boxes = []
    # 对所有slice进行遍历
    slice_nums = sorted(data_dict.keys(), key=int)
    merging_boxes = []

    for i in range(len(slice_nums)):
        slice_num = slice_nums[i]
        current_boxes = data_dict[slice_num]
        if current_boxes == [[1.0, 0.0, 0.0, 0.0, 0.0]]:
            pass
        else:
            # 第一个slice
            if (i == 0) and i+1 < len(slice_nums):
                if int(slice_nums[i+1]) == int(slice_num)+1:
                    next_boxes = data_dict[str(int(slice_num)+1)]
                    merging_boxes, merged_boxes = get_out_begin(next_boxes, current_boxes, overlap_threshold, slice_num)
                else:
                    merging_boxes, merged_boxes = get_out_single(current_boxes, overlap_threshold, slice_num)

            # 中间的slice
            elif i > 0 and i+1 < len(slice_nums):
                if int(slice_nums[i+1]) != int(slice_num)+1:
                    merging_boxes, merged_boxes = get_out_end(merging_boxes, current_boxes, overlap_threshold, slice_num)
                else:
                    next_boxes = data_dict[str(int(slice_num)+1)]
                    merging_boxes, merged_boxes = get_out_medi(merging_boxes, next_boxes, current_boxes, overlap_threshold, slice_num)

            # 最后的slice
            elif i+1 == len(slice_nums):
                merging_boxes, merged_boxes = get_out_end(merging_boxes, current_boxes, overlap_threshold, slice_num)
            
            # 将当前slice的合并信息加入到总的3D boxes列表中
            merged_3d_boxes.extend(merged_boxes)

    return merged_3d_boxes

def get_out_begin(next_boxes, current_boxes, overlap_threshold, slice_num):
    merged_boxes = []
    merging_boxes = []

    for box in current_boxes:
        merged = True
        for next_box in next_boxes:
            if calculate_overlap(box, next_box) >= overlap_threshold:
                merging_boxes.append([box + [slice_num]])
                merged = False
        if merged:
            merged_boxes.append([box + [slice_num]])

    return merging_boxes, merged_boxes

def get_out_medi(merging_boxes, next_boxes, current_boxes, overlap_threshold, slice_num):
    merged_boxes = []
    to_merge_boxes = copy.deepcopy(merging_boxes)

    for b_merging in to_merge_boxes:
        b = b_merging[-1]
        merged = True
        for box_cur in current_boxes:
            if calculate_overlap(b, box_cur) >= overlap_threshold:
                merged = False
        if merged:
            merged_boxes.append(b_merging)
            merging_boxes.remove(b_merging)

    merging_boxes_old = copy.deepcopy(merging_boxes)
    pop_index = []
    for b_c in current_boxes:
        merged_current = True
        for index, b_merging in enumerate(merging_boxes_old):
            b = b_merging[-1]
            if calculate_overlap(b, b_c) >= overlap_threshold:
                merged_current = False
                merged_next = True
                for next_box in next_boxes:
                    if calculate_overlap(b_c, next_box) >= overlap_threshold:
                        merging_boxes[index].append(b_c + [slice_num])
                        merged_next = False
                        break
                if merged_next:
                    merging_boxes[index].append(b_c + [slice_num])
                    pop_index.append(index)

        if merged_current:
            merged_next = True
            for next_box in next_boxes:
                if calculate_overlap(b_c, next_box) >= overlap_threshold:
                    merging_boxes.append([b_c + [slice_num]])
                    merged_next = False
                    break
            if merged_next:
                merged_boxes.append([b_c + [slice_num]])

    if len(pop_index) != 0:
        for dex in sorted(pop_index, reverse=True):
            merged_boxes.append(merging_boxes.pop(dex))

    return merging_boxes, merged_boxes

def get_out_end(merging_boxes, current_boxes, overlap_threshold, slice_num):
    merged_boxes = []
    pop_index = []
    for box_cur in current_boxes:
        for index, b_merging in enumerate(merging_boxes):
            b = b_merging[-1]
            if calculate_overlap(b, box_cur) >= overlap_threshold:
                merging_boxes[index].append(box_cur + [slice_num])
                break
        else:
            merged_boxes.append([box_cur + [slice_num]])
    if len(pop_index) != 0:
        for dex in sorted(pop_index, reverse=True):
            merged_boxes.append(merging_boxes.pop(dex))
    return [], merged_boxes

def get_out_single(current_boxes, overlap_threshold, slice_num):
    merged_boxes = [[box + [slice_num]] for box in current_boxes]
    return [], merged_boxes

def calculate_overlap(box1, box2):
    # 计算两个框的重叠面积
    if len(box1) == 6:
        _, x1, y1, x2, y2, _ = box1
    else:
        _, x1, y1, x2, y2 = box1
    if len(box2) == 6:
        _, x1g, y1g, x2g, y2g, _ = box2
    else:
        _, x1g, y1g, x2g, y2g = box2

    # 检查框是否完全包含
    if (x1 <= x1g <= x2 and x1 <= x2g <= x2 and
        y1 <= y1g <= y2 and y1 <= y2g <= y2) or (
        x1g <= x1 <= x2g and x1g <= x2 <= x2g and
        y1g <= y1 <= y2g and y1g <= y2 <= y2g):
        return 1.0
    else:
        xi1 = max(x1, x1g)
        yi1 = max(y1, y1g)
        xi2 = min(x2, x2g)
        yi2 = min(y2, y2g)

        inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

        box1_area = (x2 - x1) * (y2 - y1)
        box2_area = (x2g - x1g) * (y2g - y1g)

        union_area = box1_area + box2_area - inter_area
        if union_area == 0:
            return 0
        else:
            overlap = inter_area / union_area
            return overlap

merged_3d_boxes = merge_to_3d_boxes(dict_data, overlap_threshold=0.5)
print('****************************************')
for box_info in merged_3d_boxes:
    print(box_info)


****************************************
[[0.04371, 232.87, 298.83, 243.11, 305.73, '6']]
[[0.00773, 26.57, 137.67, 35.13, 149.19, '24']]
[[0.03555, 236.62, 308.99, 245.7, 317.41, '56']]
[[0.64885, 230.8, 309.5, 240.36, 317.79, '57']]
[[0.99652, 230.08, 271.7, 241.52, 278.65, '101'], [0.97974, 229.6, 271.56, 240.54, 278.25, '102'], [0.92319, 230.68, 272.27, 242.63, 278.42, '103']]
[[0.78183, 235.07, 271.58, 247.66, 277.81, '104']]
[[0.31972, 241.33, 271.13, 252.75, 277.94, '105']]
[[0.25254, 243.61, 269.38, 253.86, 275.98, '106']]
[[0.04613, 247.35, 283.63, 257.34, 291.06, '107']]
[[0.05772, 245.21, 265.76, 253.08, 272.96, '108']]
[[0.96262, 244.96, 268.78, 253.87, 276.57, '108'], [0.70568, 244.4, 269.99, 253.15, 278.21, '109'], [0.94263, 245.28, 269.1, 253.98, 276.62, '110']]
[[0.01688, 222.74, 288.91, 233.49, 297.29, '111']]
[[0.2215, 243.28, 270.5, 252.83, 279.04, '113']]
[[0.04387, 123.9, 242.9, 132.42, 249.92, '139']]
[[0.58737, 289.24, 192.37, 299.14, 200.36, '145']]
[[0.08269, 2

### Return the 3d list to 3d boxes 

In [ ]:
def convert_boxes(box_groups):
    converted_boxes = []

    for group in box_groups:
        # 跳过全零的子列表
        if len(group) == 1 and all(v == 0 for v in group[0]):
            continue
        if len(group) == 1:
            # 只有一个元素的情况
            box = group[0]
            score, x1, y1, x2, y2, slice_num = box
            z1 = int(slice_num)
            z2 = z1 + 1
            converted_boxes.append([score, x1, y1, z1, x2, y2, z2])
        else:
            # 多个元素的情况
            total_score = 0
            min_x1, min_y1, max_x2, max_y2 = float('inf'), float('inf'), 0, 0
            min_slice, max_slice = float('inf'), 0

            for box in group:
                score, x1, y1, x2, y2, slice_num = box
                total_score += score
                min_x1 = min(min_x1, x1)
                min_y1 = min(min_y1, y1)
                max_x2 = max(max_x2, x2)
                max_y2 = max(max_y2, y2)
                min_slice = min(min_slice, int(slice_num))
                max_slice = max(max_slice, int(slice_num))

            avg_score = total_score / len(group)
            z1 = min_slice
            z2 = max_slice + 1
            converted_boxes.append([round(avg_score, 5), min_x1, min_y1, z1, max_x2, max_y2, z2])

    return converted_boxes

# 调用函数并打印结果
converted = convert_boxes(merged_3d_boxes)
for box in converted:
    print(box)


### 把产生的3d的boxes转成nii文件看一下


In [155]:
import numpy as np
import os
import torch

def create_3d_array(boxes, shape):
    array_3d = np.zeros(shape, dtype=np.int32)

    for box in boxes:
        score, x1, y1, z1, x2, y2, z2 = box
        if score > 0.5:
        # array_3d[int(x1):int(x2), int(y1):int(y2), int(z1):int(z2)] = 1
            array_3d[int(y1):int(y2), int(x1):int(x2), int(z1):int(z2)] = 1

    return array_3d
# 02191204216120
# dir_nii = '/public_bme/data/xiongjl/lymph_nodes/raw_data/validation/02191204216120'
# file_name = os.listdir(dir_nii)[0]
# print(file_name)
# nii_path = os.path.join(dir_nii, file_name)
number = '870'
img_number_path = f'/public_bme/data/xiongjl/nnunet2_data/nnUNet_raw/Dataset501_lymph/imagesTr/lymph_{number}_0000.nii.gz'
label_number_path = f'/public_bme/data/xiongjl/nnunet2_data/nnUNet_raw/Dataset501_lymph/labelsTr/lymph_{number}.nii.gz'


nii_image = tio.ScalarImage(img_number_path)
shape = nii_image.shape[1:]
# 创建3D数组
array_3d = create_3d_array(converted, shape)
nii_save = tio.ScalarImage(tensor=torch.tensor(array_3d).unsqueeze(0), affine=nii_image.affine)
nii_save.save('/public_bme/data/xiongjl/Deformable-DETR_public/nii_temp/02191204216120_pred.nii.gz')
# 打印结果
# print(array_3d)


In [151]:
id_path = '/public_bme/data/xiongjl/Deformable-DETR_data_public/coco/annotations/instances_val2017_alllayers_val.json'
image_ids = get_id(id_path)
print(image_ids[0])

['02191204216120_0.jpg', '02191204216120_1.jpg', '02191204216120_2.jpg', '02191204216120_3.jpg', '02191204216120_4.jpg', '02191204216120_5.jpg', '02191204216120_6.jpg', '02191204216120_7.jpg', '02191204216120_8.jpg', '02191204216120_9.jpg', '02191204216120_10.jpg', '02191204216120_11.jpg', '02191204216120_12.jpg', '02191204216120_13.jpg', '02191204216120_14.jpg', '02191204216120_15.jpg', '02191204216120_16.jpg', '02191204216120_17.jpg', '02191204216120_18.jpg', '02191204216120_19.jpg', '02191204216120_20.jpg', '02191204216120_21.jpg', '02191204216120_22.jpg', '02191204216120_23.jpg', '02191204216120_24.jpg', '02191204216120_25.jpg', '02191204216120_26.jpg', '02191204216120_27.jpg', '02191204216120_28.jpg', '02191204216120_29.jpg', '02191204216120_30.jpg', '02191204216120_31.jpg', '02191204216120_32.jpg', '02191204216120_33.jpg', '02191204216120_34.jpg', '02191204216120_35.jpg', '02191204216120_36.jpg', '02191204216120_37.jpg', '02191204216120_38.jpg', '02191204216120_39.jpg', '02191204